In [21]:
from data.dataset import EmotionsDataset

train_dataset = EmotionsDataset(split="train")
valid_dataset = EmotionsDataset(split="valid")
test_dataset = EmotionsDataset(split="test")

No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (C:/Users/ryans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)
No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (C:/Users/ryans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)
No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (C:/Users/ryans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


In [22]:
print(train_dataset[0])
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

("My favourite food is anything I didn't have to cook myself.", 13)
43410
5426
5427


In [23]:
def get_class_weights(train_labels):
    label_distribution = [0 for i in range(14)]

    for label in train_labels:
        label_distribution[label] += 1

    class_weights = dict()

    for i, label_num in enumerate(label_distribution):
        class_weights[i] = label_num/len(train_labels)

    return class_weights

In [24]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download, pos_tag
download('punkt')
download('wordnet')
download('omw-1.4')
download('averaged_perceptron_tagger')
download('stopwords')

from collections import defaultdict

def lem_text(text):
    text = [word.lower() for word in text]
    text = [word_tokenize(word) for word in text]

    word_tags = defaultdict(lambda: wordnet.NOUN)
    word_tags['J'] = wordnet.ADJ
    word_tags['V'] = wordnet.VERB
    word_tags['R'] = wordnet.ADV

    for i, words in enumerate(text):
        lemmed_text = []
        lemmer = WordNetLemmatizer()

        for word, tag in pos_tag(words):
            if word not in stopwords.words('english') and word.isalpha():
                lem_word = lemmer.lemmatize(word, word_tags[tag[0]])
                lemmed_text.append(lem_word)

        text[i] = str(lemmed_text)

    return text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_dataset.x = lem_text(train_dataset.x)
valid_dataset.x = lem_text(valid_dataset.x)
test_dataset.x = lem_text(test_dataset.x)

tfidf_vectoriser = TfidfVectorizer()
tfidf_vectoriser.fit(train_dataset.x)

train_dataset.x = tfidf_vectoriser.transform(train_dataset.x)
valid_dataset.x = tfidf_vectoriser.transform(valid_dataset.x)
test_dataset.x = tfidf_vectoriser.transform(test_dataset.x)#

weights = get_class_weights(train_dataset.y)

TypeError: unsupported operand type(s) for /: 'int' and 'list'

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

svm_linear = svm.SVC(kernel='linear', class_weight=weights)
svm_linear.fit(train_dataset.x, train_dataset.y)

NameError: name 'weights' is not defined

In [ ]:
preds = svm_linear.predict(test_dataset.x)
acc = accuracy_score(test_dataset.y, preds)*100
print("Test Accuracy: ", acc)

Test Accuracy:  48.000737055463425


In [ ]:
svm_poly = svm.SVC(kernel='poly')
svm_poly.fit(train_dataset.x, train_dataset.y)

SVC(kernel='poly')

In [ ]:
preds = svm_poly.predict(test_dataset.x)
acc = accuracy_score(test_dataset.y, preds)*100
print("Test Accuracy: ", acc)

Test Accuracy:  39.063939561452


In [ ]:
svm_sig = svm.SVC(kernel='sigmoid')
svm_sig.fit(train_dataset.x, train_dataset.y)

SVC(kernel='sigmoid')

In [ ]:
preds = svm_sig.predict(test_dataset.x)
acc = accuracy_score(test_dataset.y, preds)*100
print("Test Accuracy: ", acc)

Test Accuracy:  54.43154597383453


In [ ]:
svm_rbf = svm.SVC(kernel='rbf')
svm_rbf.fit(train_dataset.x, train_dataset.y)

SVC()

In [ ]:
preds = svm_rbf.predict(test_dataset.x)
acc = accuracy_score(test_dataset.y, preds)*100
print("Test Accuracy: ", acc)

Test Accuracy:  54.67108899944721
